Time Series Forecast with NHiTS on the Vierlinden dataset (all sensors, 2021)

In [1]:
import sys
sys.path.append('./pytorch-forecasting/')
import pandas as pd
import torch
import lightning.pytorch as pl
from pytorch_forecasting import NHiTS, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
pl.seed_everything(42)

Global seed set to 42


42

### Load dataset

In [2]:
# Read the dataset into a DataFrame
data = pd.read_csv('./RIWWER/Vierlinden/Vierlinden_2021_All.csv')

# Drop columns that have lots of missing values
data.drop(["FLP_Hohenstand_Pumpensumpf_pval","FLP_Strom_P3_pval","FLP_Strom_P4_pval","FLP_Strom_P5_pval","Durchfluss SWP1 und SWP2_pval","FLP_Hohenstand_Becken1_pval","FLP_Hohenstand_Becken3_pval","FLP_Hohenstand_Beckne2_pval"], axis=1, inplace=True)

# NaNs are not allowed by the model
data.fillna(method="bfill", inplace=True)
data.fillna(method="ffill", inplace=True)

# Set Datetime as index
data['Datetime'] = pd.to_datetime(data['Datetime'])

# One time series for the whole year?
data['series'] = 0

# As many timesteps per timeseries as hours in every month?
time_idx = []
for i in range(1):
    timesteps = len( data[ data['series'] == i ] )
    time_idx += list(range(timesteps))
data['time_idx'] = time_idx

In [3]:
data.head()

,Datetime,Schieber Position_pval,Oberwasser_pval,Unterwasser_pval,Durchflumenge_pval,Berechnete Durchflussmenge_pval,Fllstand SWS_pval,Fllstand RWS_pval,Strom P1_pval,Strom P2_pval,...,Strom Pumpe 2_pval1,Strom Pumpe 3_pval,Niederschlag,Füllstand_RRB,Entleerung_RüB,Füllstand_RüB_1,Füllstand_RüB_2,Füllstand_RüB_3,series,time_idx
0,2021-01-01 00:00:00,100.0,8.140845,5.753623,7.689189,7.732558,75.717949,36.0,1.076923,0.0,...,0.0,1.0,0.000000,1.47,0.098,3.16,3.08,2.72,0,0
1,2021-01-01 01:00:00,100.0,8.000000,5.173913,6.808219,8.271739,75.717949,36.0,1.076923,0.0,...,0.0,1.0,1.182353,1.47,0.099,3.16,3.08,2.72,0,1
2,2021-01-01 02:00:00,100.0,7.967742,5.000000,5.813333,7.197674,75.717949,36.0,1.076923,0.0,...,0.0,1.0,1.182353,1.47,0.096,3.16,3.08,2.72,0,2
3,2021-01-01 03:00:00,100.0,7.076923,4.843750,4.216216,4.743243,75.717949,36.0,1.076923,0.0,...,0.0,1.0,1.182353,1.47,0.098,3.16,3.08,2.72,0,3
4,2021-01-01 04:00:00,100.0,8.464789,5.466667,8.384615,8.325000,75.717949,36.0,1.076923,0.0,...,0.0,1.0,1.182353,1.47,0.098,3.16,3.08,2.72,0,4


In [4]:
# Parameters for dataloaders
max_encoder_length = 24*2
max_prediction_length = 5*2
training_cutoff = data["time_idx"].max() * 4 // 5 # 80% for training
context_length = max_encoder_length
prediction_length = max_prediction_length
batch_size = 32

In [5]:
# Load best model (from NHits_Vierlinden_Train.ipynb)
best_model_path = './RIWWER/torch_forecasting/model_checkpoints/NHits_Vierlinden_saved_datasets/lightning_logs/version_2/checkpoints/epoch=14-step=2250.ckpt'
best_model = NHiTS.load_from_checkpoint(best_model_path)

/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:196: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:196: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


### Corrupt sensor clusters

In [6]:
# Sensor clusters
# Weather and rain tanks: ['Niederschlag', 'Füllstand_RRB', 'Füllstand_RüB_1', 'Füllstand_RüB_2', 'Füllstand_RüB_3']
sensors = {
    "Herzogstr":     ['Schieber Position_pval', 'Oberwasser_pval', 'Unterwasser_pval', 'Durchflumenge_pval', 'Berechnete Durchflussmenge_pval'],
    "Kaiserstr":     ['Fllstand SWS_pval', 'Fllstand RWS_pval', 'Strom P1_pval', 'Strom P2_pval', 'Strom P3_pval', 'Strom P4_pval', 'Strom P5_pval', 'Strom P6_pval'],
    "Kreuzweg":      ['Fllstand Pumpensumpf_pval', 'Strom Pumpe 1_pval', 'Strom Pumpe 2_pval'],
    "Vierlindenhof": ['Fllstand Pumpensumpf_pval1', 'Strom Pumpe 1_pval1', 'Strom Pumpe 2_pval1', 'Strom Pumpe 3_pval']
}

In [7]:
# Turn off sensors iteratively according to the cluster location
for cluster in sensors:
    print("\n\n++++++++++ Turning off " + cluster + " ++++++++++\n")

    ### Create Dataloader for corrupted dataset ###
    validation = TimeSeriesDataSet(
        data[lambda x: x.time_idx > training_cutoff],
        target_normalizer="auto",
        time_idx="time_idx",
        target="Entleerung_RüB",
        categorical_encoders={"series": NaNLabelEncoder().fit(data.series)},
        group_ids=["series"],
        time_varying_unknown_reals=list(set(data.columns) - {'Datetime', 'series', 'time_idx'}),
        max_encoder_length=context_length,
        min_encoder_length=max_encoder_length,
        max_prediction_length=prediction_length,
        min_prediction_length=max_prediction_length,
        allow_missing_timesteps=True
    )
    loaded_validation = validation.load('./RIWWER/Vierlinden/val_set')

    # Note: After replacing the feature with zeros, the TimeSeriesDataset applies 'transform_values'
    #       which leads to all the values in that column being replaced not by 0, but by another rescaled value
    #       You may want to comment that line out in timeseries.py
    for column in sensors[cluster]:
        loaded_validation.set_overwrite_values(0.0, column, 'all')
    # TODO: only one single value can be overwritten now.

    val_dataloader = loaded_validation.to_dataloader(train=False, batch_size=batch_size, num_workers=18)
    break
    ### Make prediction and evaluate ###
    actuals = torch.cat([y[0] for x, y in iter(val_dataloader)]).to(torch.device('cuda:0'))
    predictions = best_model.predict(val_dataloader,
                                     trainer_kwargs=dict(default_root_dir="./RIWWER/torch_forecasting/model_checkpoints/NHits_Vierlinden_saved_datasets"))
    err = actuals - predictions
    mae = err.abs().mean()
    print('MAE = ' + str(mae))
    rmse = torch.sqrt( torch.square(err).mean() )
    print('RMSE = ' + str(rmse))



++++++++++ Turning off Herzogstr ++++++++++



/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not ha

In [8]:
for x, y in iter(val_dataloader):
    print(x['encoder_cont'][0][0])

tensor([ 0.0927, -1.2365, -0.1461, -0.2922, -0.0663,  0.1037, -0.4271, -0.0533,
        -0.5470, -0.6442, -0.2640, -0.2755, -0.3890, -0.1162, -0.0157, -0.3383,
        -0.1735, -0.0268, -0.0575,  0.0293, -0.4598, -0.3833, -0.5429, -0.3601,
        -0.0550, -1.0269])
tensor([-0.2787,  0.0594, -0.1461, -0.2922, -0.0663,  0.1037, -0.4086, -0.0533,
         0.0884, -0.6442, -0.2561, -0.1342, -0.3890, -0.1162, -0.1533, -0.0316,
        -0.1735, -0.0268, -0.0575,  0.0293, -0.0612, -0.3833, -0.5429, -0.3601,
        -0.0550, -0.0829])
tensor([-0.5579, -0.1614, -0.1461, -0.2922, -0.0663, -0.0877, -0.4448, -0.0533,
         0.4572, -0.6442, -0.2561,  0.0747, -0.3792, -0.1162, -0.4164, -0.0828,
        -0.1735, -0.0268, -0.0575,  0.0293,  0.1880, -0.3833, -0.5429, -0.3601,
        -0.0550, -0.6139])
tensor([-0.6836, -0.2654, -0.1461, -0.2922, -0.0663, -0.0877, -0.3593, -0.0533,
         0.1006, -0.6442, -0.2561, -0.0879, -0.3695, -0.1076, -0.1362, -0.3383,
        -0.1735, -0.0268, -0.0575,  0.0

In [9]:
loaded_validation._overwrite_values

{'values': tensor(-0.6442, dtype=torch.float64),
 'variable': 'Berechnete Durchflussmenge_pval',
 'target': 'all'}